# Classifing digits from the MNIST-dataset

In [ ]:
import numpy as np
np.random.seed(123)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import TensorBoard

In [ ]:
from keras import backend as K
K.set_image_dim_ordering('th')

## Load the dataset

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
print("Train: ", X_train.shape)
print("Test : ", X_test.shape)

Let's print some examples from the training dataset

In [ ]:
train_samples = np.random.randint(X_train.shape[0], size=5)
fig = plt.figure()
for i, s in enumerate(train_samples):
    sub_fig = fig.add_subplot(1, 5, i + 1)
    f = plt.imshow(X_train[s], cmap='gray')
    f.axes.get_xaxis().set_visible(False)
    f.axes.get_yaxis().set_visible(False)
    sub_fig.set_title(y_train[s])
plt.show()

### Prepare the dataset

In [ ]:
# Reshape data to contain depth (which would be necessary for RGB-images)
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
print(X_train.shape)

In [ ]:
# Convert all data to float and normalize to be in range [0,1]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
# convert numbers for vectors for categorization by Keras
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
print Y_train.shape

### Define model architecture

In [ ]:
model = Sequential()
 
model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(1,28,28)))
model.add(Convolution2D(32, 3, 3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

### Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

### Fit the model on training data

You can optionally enable callbacks for Tensorboard to further investigate on learning rate, etc.

In [ ]:
tb_callback = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

In [ ]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=2, verbose=1, callbacks=[tb_callback])

### Evaluate the model

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
print(score)

### Use model to predict some digits

In [ ]:
num_test_examples = 32
per_row = 8

# sample some digits from the test-dataset
test_samples = np.random.randint(X_test.shape[0], size=num_test_examples)

# predict digit
prediction = model.predict(X_test[test_samples], batch_size=num_test_examples)

fig = plt.figure()
for i, s in enumerate(test_samples):    
    predited_digit = np.argmax(prediction[i])

    sub_fig = fig.add_subplot(1 + (num_test_examples / per_row), per_row, i + 1)
    f = plt.imshow(X_test[s].reshape(28, 28), cmap='gray')    
    f.axes.get_xaxis().set_visible(False)
    f.axes.get_yaxis().set_visible(False)
    sub_fig.set_title(predited_digit)
    
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
